## Data Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

In [2]:
property_data = pd.read_csv("data/nsw_property_data_2019-09-10.csv")
property_data.shape

(57680, 10)

In [3]:
property_data.head()

,link,address,postcode,price,num_beds,num_baths,num_cars,property_area,land_area,property_type
0,https://www.realestate.com.au/property-apartme...,"511/188 Day Street, Sydney",2000,For Sale | Obsidian Property,3,2,1,None,None,Apartment
1,https://www.realestate.com.au/property-apartme...,"1803/178 Thomas Street, Haymarket",2000,Contact Agent,2,2,2,None,None,Apartment
2,https://www.realestate.com.au/property-apartme...,"2406/168 Kent Street, Sydney",2000,"$1,540,000",1,1,1,None,None,Apartment
3,https://www.realestate.com.au/property-apartme...,"806/1 Hosking Place, Sydney",2000,"$719,000",1,1,None,None,None,Apartment
4,https://www.realestate.com.au/property-apartme...,"1702/11-15 Alberta Street, Sydney",2000,Auction,2,2,1,None,None,Apartment


Strip whitespace from number of bed/bath/car and convert to integer

In [4]:
for col in ['num_beds','num_baths','num_cars']:
    property_data[col] = np.where(property_data[col]=='None',0,property_data[col]).astype(int)

Convert price to a numeric if it exists

In [5]:
property_data['price_strip'] = (property_data['price'].apply(lambda x: x.replace('$','')) 
    .apply(lambda x: x.replace(',','')).apply(lambda x: x.replace(r'[a-zA-Z]','')))

property_data['price_numeric'] = pd.to_numeric(property_data['price_strip'], errors='coerce')

In [6]:
print("% of all listings which have unknown price: ",round(100*property_data['price_numeric'].isnull().sum()/len(property_data),2))

% of all listings which have unknown price:  48.07


Create flag for Auction if price contains 'Auction'

In [7]:
property_data['auction'] = np.where(property_data.price.str.lower().str.contains('auction'),True,False)

Property area and land area to numeric

In [8]:
for col in ['property_area','land_area']:
    property_data[col] = property_data[col].apply(lambda x: x.strip())
    property_data[col+'_num'] = pd.to_numeric(property_data[col], errors='coerce')

Join the latitude and longitudes to the postcode (we could get even more exact lat/longs from a geocoding service)

In [9]:
# Get Postcodes
postcodes = pd.read_csv("./data/australian_postcodes.csv")
postcode_mapping = postcodes[(postcodes.type=="Delivery Area") & (postcodes.long != postcodes.lat)]

# We just need 1 row per postcode
postcode_mapping= postcode_mapping[['postcode','lat','long']].drop_duplicates()

In [10]:
property_data_postcode = property_data.merge(right=postcode_mapping, how='left', on='postcode')

In [11]:
property_data_postcode.shape

(57680, 17)

In [12]:
property_data

,link,address,postcode,price,num_beds,num_baths,num_cars,property_area,land_area,property_type,price_strip,price_numeric,auction,property_area_num,land_area_num
0,https://www.realestate.com.au/property-apartme...,"511/188 Day Street, Sydney",2000,For Sale | Obsidian Property,3,2,1,None,None,Apartment,For Sale | Obsidian Property,NaN,False,NaN,NaN
1,https://www.realestate.com.au/property-apartme...,"1803/178 Thomas Street, Haymarket",2000,Contact Agent,2,2,2,None,None,Apartment,Contact Agent,NaN,False,NaN,NaN
2,https://www.realestate.com.au/property-apartme...,"2406/168 Kent Street, Sydney",2000,"$1,540,000",1,1,1,None,None,Apartment,1540000,1540000.0,False,NaN,NaN
3,https://www.realestate.com.au/property-apartme...,"806/1 Hosking Place, Sydney",2000,"$719,000",1,1,0,None,None,Apartment,719000,719000.0,False,NaN,NaN
4,https://www.realestate.com.au/property-apartme...,"1702/11-15 Alberta Street, Sydney",2000,Auction,2,2,1,None,None,Apartment,Auction,NaN,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57675,https://www.realestate.com.au/property-townhou...,"Address available on request, Moncrieff",2914,"$440,000",2,2,1,100,None,Townhouse,440000,440000.0,False,100.0,NaN
57676,https://www.realestate.com.au/property-townhou...,"Address available on request, Moncrieff",2914,"$465,000",3,2,1,107,None,Townhouse,465000,465000.0,False,107.0,NaN
57677,https://www.realestate.com.au/property-house-a...,"Address available on request, Moncrieff",2914,"Offers Above $350,000",3,2,2,None,None,House,Offers Above 350000,NaN,False,NaN,NaN
57678,https://www.realestate.com.au/property-residen...,"11/1 Jack Ryan Street, Forde",2914,Auction,0,0,0,None,0.58,Residential Land,Auction,NaN,True,NaN,0.58


In [13]:
property_data_postcode.to_csv('data/property_data_cleaned_21092019.csv')